In [15]:
#from pyspark import SparkConf, SparkContext
#conf = SparkConf().setAppName('spark-yarn') \
#                .setMaster('spark://172.22.0.2:7077')
#sc = SparkContext(conf=conf)

In [16]:
%ls /opt/spark-data/libs/

environment/  environment.tar.gz  pyspark-xgboost.zip  xgb_0_9/  xgb_1/


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [18]:
spark = SparkSession.builder.config('spark.executor.memory', '4g') \
                    .config('spark.jars','/opt/spark-data/libs/xgb_0_9/*') \
                    .master("spark://spark-master:7077").getOrCreate()                    

In [19]:
# testing the repackaging - failed for now - June 2020
#spark.sparkContext.addPyFile("/opt/spark-data/libs/xgboost4j-spark_2.12-1.0.0.jar")
#spark.sparkContext.addPyFile("/opt/spark-data/libs/xgboost4j_2.12-1.0.0.jar")

In [20]:
# 0.9 wrapper
spark.sparkContext.addPyFile("/opt/spark-data/libs/pyspark-xgboost.zip")

In [21]:
import sparkxgb

## Iris Example

In [22]:
import pyspark
pyspark.__version__

'2.4.5'

In [23]:

# create schema type for dataset
schema = StructType([
            StructField("sepal length", DoubleType(), True),
            StructField("sepal width", DoubleType(), True),
            StructField("petal length", DoubleType(), True),
            StructField("petal width", DoubleType(), True),
            StructField("class", StringType(), True)
        ])

# load in iris dataset
rawInput = spark.read.schema(schema).csv("/opt/spark-data/iris.data")

# convert text class to index
stringIndexer = StringIndexer(inputCol="class", outputCol="classIndex")

In [24]:
model = stringIndexer.fit(rawInput)

In [25]:
# transform labels
labelTransformed = model.transform(rawInput).drop("class")

# xgb spark requires 
vectorAssembler = VectorAssembler(inputCols=["sepal length", "sepal width", "petal length", "petal width"],
                                  outputCol="features")

xgbInput = vectorAssembler.transform(labelTransformed)
xgbInput = xgbInput.select("features", "classIndex")

train, test = xgbInput.randomSplit([0.8,0.2])


In [26]:
train

DataFrame[features: vector, classIndex: double]

In [27]:
xgb_model = sparkxgb.XGBoostClassifier(
    featuresCol="features",
    labelCol="classIndex"
)

Py4JJavaError: An error occurred while calling None.ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.
: java.lang.NoSuchMethodError: org.apache.spark.ml.param.shared.HasWeightCol.$init$(Lorg/apache/spark/ml/param/shared/HasWeightCol;)V
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.<init>(XGBoostClassifier.scala:43)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.<init>(XGBoostClassifier.scala:48)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCom

In [ ]:
xgb_model.setParams(eta=0.1,
                   maxDepth=2,
                   objective="multi:softprob",
                   numClass=3,
                   numRound=10,
                   numWorkers=2)

model = xgb_model.fit(train)

In [ ]:
results = model.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",
                                              labelCol="classIndex",
                                              metricName="weightedPrecision")

# compute the classification error on test data.
accuracy = evaluator.evaluate(results)
print("Test Error = %g" % (1.0 - accuracy))

## Kaggle Titanic Example

In [28]:
from pyspark.sql.types import *
schema = StructType(
  [StructField("PassengerId", DoubleType()),
    StructField("Survival", DoubleType()),
    StructField("Pclass", DoubleType()),
    StructField("Name", StringType()),
    StructField("Sex", StringType()),
    StructField("Age", DoubleType()),
    StructField("SibSp", DoubleType()),
    StructField("Parch", DoubleType()),
    StructField("Ticket", StringType()),
    StructField("Fare", DoubleType()),
    StructField("Cabin", StringType()),
    StructField("Embarked", StringType())
  ])

In [29]:
df_raw = spark\
  .read\
  .option("header", "true")\
  .schema(schema)\
  .csv("/opt/spark-data/train.csv")

In [30]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [31]:
df = df_raw.na.fill(0)

In [32]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

In [33]:
vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

In [36]:
from pyspark.ml import Pipeline

In [35]:
import sparkxgb
xgboost = sparkxgb.XGBoostClassifier(

)

Py4JJavaError: An error occurred while calling None.ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.
: java.lang.NoSuchMethodError: org.apache.spark.ml.param.shared.HasWeightCol.$init$(Lorg/apache/spark/ml/param/shared/HasWeightCol;)V
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.<init>(XGBoostClassifier.scala:43)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.<init>(XGBoostClassifier.scala:48)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCom

In [ ]:
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)

In [ ]:
trainDF.head(2)

In [ ]:
pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, xgboost])

In [ ]:
model = xgboost.fit(trainDF)

### Shutdown

In [14]:
spark.stop()